In [1]:
#Установка ludwig
!pip install ludwig==0.4.1


[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Установка transformers
!pip install transformers


[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
from itertools import groupby

In [4]:
#import необходимых модулей

import pandas as pd
from ludwig.api import LudwigModel
from ludwig.visualize import learning_curves
import yaml
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import numpy as np
from datetime import datetime, timedelta

In [5]:
#Считывание данных в DataFrame 

train= pd.read_csv('train_predobr.csv', sep=';', index_col=None)
test= pd.read_csv('test_predobr.csv', sep=';', index_col=None)
tabliza= pd.read_csv('tabliza.csv', sep=';', index_col=None)

In [6]:
train=train.merge(tabliza, how="left", on="overall_worklogs")

In [7]:
train

,id,project_id,assignee_id,creator_id,Number_in_project,summary_to_eng,Year,Month,Day,Hour,DayOfWeek,Id_razn,mean_words_comment,count_comments,overall_worklogs,overall_worklogs_num,id|Количество
0,819952,5,93,93,10678,UI tests commissioned by Dobro KG,2019,10,1,5,2,0,0.000000,0,1800,25,615
1,819949,5,93,93,10679,"UI tests of the ""Profile"" section",2019,10,1,5,2,0,4.000000,1,7200,96,646
2,819947,5,93,93,10680,"UI tests of the section ""Personal account""",2019,10,1,6,2,0,0.000000,0,14400,189,224
3,819943,5,93,93,10682,"UI tests of the ""News"" section",2019,10,1,6,2,0,0.000000,0,900,12,276
4,819941,5,93,93,10683,"UI tests of the ""Zones of discounts and surcha...",2019,10,1,6,2,0,2.000000,1,900,12,276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9584,688414,48,1,116,779,Make a test to check teachers pages for VN & TH,2020,6,15,9,1,0,0.000000,0,26040,290,1
9585,670934,56,2,2,1,Set up the neo4j by using a dataset from Slack,2020,6,2,10,2,72,17.000000,3,16200,205,108
9586,670932,56,2,2,3,Set up a graph in neo4j by GitLab dataset,2020,6,5,9,5,0,0.000000,0,3600,51,784
9587,670931,56,2,2,4,Parse a dataset from teacherly slack,2020,6,5,13,5,0,0.000000,0,6300,86,86


In [8]:
test

,mean_words_comment,count_comments,id,project_id,assignee_id,creator_id,Number_in_project,Year,Month,Day,Hour,DayOfWeek,Id_razn,summary_to_eng
0,62.000000,8,675975,5,112,112,11140,2020,1,15,9,3,478,Download all mp3 files of calls to disk
1,16.500000,4,675972,5,112,112,11141,2020,1,15,9,3,721,Delete mp3 call files from the server before 0...
2,7.000000,1,675965,5,94,54,11144,2020,1,16,4,4,0,Remove the tariff selection window and transfe...
3,41.500000,2,675961,5,94,94,11146,2020,1,16,11,4,180,"During authorization, the user/driver does not..."
4,47.571429,7,675955,5,1,54,11150,2020,1,17,5,5,3367,The main problems for corp. department (as of ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065,3.000000,1,702545,48,1,1,1392,2020,9,24,11,4,0,Order to fix: audit result & save root cause
1066,12.600000,10,702528,48,1,1,1402,2020,9,28,14,1,33,Lỗi EPC không tải trang được
1067,0.000000,0,702499,48,1,1,1422,2020,10,6,13,2,0,Line red cannot calling out from system: Audit...
1068,0.000000,0,702376,48,1,1,1493,2020,11,5,15,4,0,ipphone problem: push & audit result


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9589 entries, 0 to 9588
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    9589 non-null   int64  
 1   project_id            9589 non-null   object 
 2   assignee_id           9589 non-null   int64  
 3   creator_id            9589 non-null   int64  
 4   Number_in_project     9589 non-null   int64  
 5   summary_to_eng        9589 non-null   object 
 6   Year                  9589 non-null   int64  
 7   Month                 9589 non-null   int64  
 8   Day                   9589 non-null   int64  
 9   Hour                  9589 non-null   int64  
 10  DayOfWeek             9589 non-null   int64  
 11  Id_razn               9589 non-null   int64  
 12  mean_words_comment    9589 non-null   float64
 13  count_comments        9589 non-null   int64  
 14  overall_worklogs      9589 non-null   int64  
 15  overall_worklogs_num 

In [10]:
train.columns

Index(['id', 'project_id', 'assignee_id', 'creator_id', 'Number_in_project',
       'summary_to_eng', 'Year', 'Month', 'Day', 'Hour', 'DayOfWeek',
       'Id_razn', 'mean_words_comment', 'count_comments', 'overall_worklogs',
       'overall_worklogs_num', 'id|Количество'],
      dtype='object')

In [45]:

config = {
    "input_features": [
        {
            "name": "project_id",
            "type": "category",      
        },
        {
            "name": "assignee_id",
            "type": "category",      
        },
        {
            "name": "creator_id",
            "type": "category",      
        },
        {
            "name": "summary_to_eng",
            "type": "text", 
            "encoder": "bert",
        },        

         {
            "name": "Year",
            "type": "category",      
        },   
        {
            "name": "Month",
            "type": "category",      
        },          
         {
            "name": "Hour",
            "type": "category",      
        },   
        {
            "name": "Day",
            "type": "category",      
        },        
        {
            "name": "DayOfWeek",
            "type": "category",      
        },   
        {
            "name": "Id_razn",
            "type": "numerical", 
            "normalization":"zscore"
        },          
        {
            "name": "mean_words_comment",
            "type": "numerical", 
            "normalization":"zscore"
        },           
        {
            "name": "count_comments",
            "type": "numerical", 
            "normalization":"zscore"
        },             
                
    ],
    "output_features": [
        {
            "name": 'overall_worklogs_num',
            "type": "numerical",
            "loss":
                    {"type": 'mean_squared_error'},
            #"clip": [0,7000]
            
            
            
        }
    ],
        'training': {
        'batch_size': 8,
        'decay': True,     
        'trainable': True,
        'learning_rate': 0.005,
        'epochs': 100,
        'early_stop': 20,
        'validation_field': 'PATIENT_ID_COUNT',
        'validation_metric': 'loss',
        'learning_rate_warmap_epoch': 1,
        
    },
        'preprocessing':{
        'split_probabilities': [0.8, 0.1, 0.1]}
}

In [ ]:
vlad = LudwigModel(config, logging_level=50,)

In [ ]:
%%time
print("Training Model...")
train_stats_vlad, _, _ = vlad.train(
    train,   
    model_name='vlad',
    skip_save_processed_input=True,
    random_seed=69)

In [ ]:
learning_curves(train_stats_vlad, output_feature_name='label', model_names='vlad')

In [ ]:
predictions, _ = vlad.predict(test)
predictions['overall_worklogs_num_predictions']

In [34]:
#Получение ответов
y_pred_solution=predictions['overall_worklogs_num_predictions'].values
y_pred_solution_plus=np.round(y_pred_solution, 0)
y_pred_solution_round=y_pred_solution_plus.astype(int)
y_pred_solution_round


In [37]:
# Заводим результаты в тест
test['overall_worklogs_num'] = y_pred_solution_round

test

In [38]:
# Просматриваем результаты
test['overall_worklogs_num'].min()

In [39]:
test['overall_worklogs_num'].max()

In [40]:
# Клипуем выходящих за предел интервала (снизу, так как заходы вверх не исключены)
test.loc[(test['overall_worklogs_num']<0),'overall_worklogs_num']=5

In [41]:
test=test.merge(tabliza, how="left", on="overall_worklogs_num")

In [ ]:
test

In [ ]:
test_unit=test[['id','overall_worklogs']]

In [ ]:
test_unit.to_csv('предварительный вариант проверка Ludwig.csv', sep=',', index=None)